In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import random

In [3]:
pd.__version__

'1.4.4'

Se a gente lê o arquivo de 3GB inteiro, vai dar problema. Aqui tem uma receita para ler somente x%: https://www.kaggle.com/questions-and-answers/53925 

## Selecionar apenas uma amostra aleatória de 1% dos alunos

In [4]:
def skip(i,fraction=0.01):
    if i == 0:
        return False  # precisamos manter o row=0 para ter o header
    else:
        return random.random() > fraction

[skip(i) for i in range(10)]

[False, True, True, True, True, True, True, True, True, True]

In [6]:
%time df = pd.read_csv('../inep/enem/2019/DADOS/MICRODADOS_ENEM_2019.csv',skiprows = skip, encoding='latin1',sep=";")

CPU times: user 23.5 s, sys: 1.55 s, total: 25 s
Wall time: 25 s


In [7]:
df

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004666,2019,1500602,Altamira,15,PA,23,F,1,2,...,A,A,A,B,A,A,C,A,A,A
1,190001004673,2019,1501204,Baião,15,PA,22,M,1,1,...,A,A,A,B,B,A,B,B,A,A
2,190001004681,2019,1501808,Breves,15,PA,23,M,1,3,...,A,A,A,B,B,A,B,A,A,A
3,190001004775,2019,1501402,Belém,15,PA,27,M,1,2,...,A,A,A,B,A,A,B,A,B,A
4,190001004999,2019,1502756,Concórdia do Pará,15,PA,19,F,1,3,...,A,A,A,B,A,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51041,190006119131,2019,4313409,Novo Hamburgo,43,RS,18,M,1,1,...,B,A,A,C,A,A,D,A,B,B
51042,190006119172,2019,5101704,Barra do Bugres,51,MT,20,M,1,3,...,A,A,A,B,A,A,C,B,C,B
51043,190006119550,2019,1302603,Manaus,13,AM,19,F,1,3,...,A,A,A,B,A,A,E,A,B,B
51044,190006119735,2019,2211001,Teresina,22,PI,19,F,1,2,...,A,A,A,B,A,A,D,B,A,B


In [146]:
#df.describe()

In [145]:
#df.sample(5)

In [10]:
df["TP_PRESENCA_CN"].value_counts()

1    37041
0    13985
2       20
Name: TP_PRESENCA_CN, dtype: int64

## Limpeza dos dados, selecionando apenas alunos que não faltaram na prova

In [13]:

def filterdf(df):
    df = df[df["TP_PRESENCA_CN"] == 1] #presente nas 4 provas
    df = df[df["TP_PRESENCA_CH"] == 1]
    df = df[df["TP_PRESENCA_LC"] == 1]
    df = df[df["TP_PRESENCA_MT"] == 1]
    df = df[df["IN_TREINEIRO"] == 0] # não é treineiro
    df = df[df["TP_ST_CONCLUSAO"].isin([1,2])] #afirma que concluiu ou vai concluir EM em 2019
    # não queremos as notas == 0
    df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df

dff = filterdf(df)
dff

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004666,2019,1500602,Altamira,15,PA,23,F,1,2,...,A,A,A,B,A,A,C,A,A,A
1,190001004673,2019,1501204,Baião,15,PA,22,M,1,1,...,A,A,A,B,B,A,B,B,A,A
2,190001004681,2019,1501808,Breves,15,PA,23,M,1,3,...,A,A,A,B,B,A,B,A,A,A
3,190001004775,2019,1501402,Belém,15,PA,27,M,1,2,...,A,A,A,B,A,A,B,A,B,A
4,190001004999,2019,1502756,Concórdia do Pará,15,PA,19,F,1,3,...,A,A,A,B,A,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51038,190006118887,2019,3305802,Teresópolis,33,RJ,20,F,1,1,...,A,A,A,B,A,A,D,A,A,B
51039,190006119047,2019,3545803,Santa Bárbara d'Oeste,35,SP,18,F,1,3,...,B,A,B,C,A,A,D,B,C,A
51041,190006119131,2019,4313409,Novo Hamburgo,43,RS,18,M,1,1,...,B,A,A,C,A,A,D,A,B,B
51042,190006119172,2019,5101704,Barra do Bugres,51,MT,20,M,1,3,...,A,A,A,B,A,A,C,B,C,B


In [16]:
import inspect
filter_conditions = inspect.getsource(filterdf)
print(filter_conditions)

def filterdf(df):
    df = df[df["TP_PRESENCA_CN"] == 1] #presente nas 4 provas
    df = df[df["TP_PRESENCA_CH"] == 1]
    df = df[df["TP_PRESENCA_LC"] == 1]
    df = df[df["TP_PRESENCA_MT"] == 1]
    df = df[df["IN_TREINEIRO"] == 0] # não é treineiro
    df = df[df["TP_ST_CONCLUSAO"].isin([1,2])] #afirma que concluiu ou vai concluir EM em 2019
    # não queremos as notas == 0
    df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df



## Salvar a amostra filtrada em disco

In [19]:

fn = "data/enem19-1perc"
dff.to_csv(f'{fn}.csv',index=False)
with open(fn+'-filter_conditions.txt','w') as f:
    f.write(filter_conditions)

## Selecionar apenas as colunas que desejamos trabalhar

In [165]:

def selecao(dff):
    df_selecao = dff[["NU_INSCRICAO","NU_ANO","CO_MUNICIPIO_RESIDENCIA","NO_MUNICIPIO_RESIDENCIA","CO_UF_RESIDENCIA","SG_UF_RESIDENCIA","TP_COR_RACA","TP_SEXO","NU_IDADE","TP_ENSINO","TP_ESCOLA","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]
    return df_selecao

    

In [167]:
df_selecionado = selecao(df)
df_selecionado

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
0,190001004658,2019,1502939,Dom Eliseu,15,PA,3,F,17,NaN,1,506.0,495.1,458.9,469.5,640.0
1,190001004685,2019,1503606,Itaituba,15,PA,0,M,25,NaN,1,NaN,NaN,NaN,NaN,NaN
2,190001004894,2019,1501402,Belém,15,PA,2,F,23,1.0,1,NaN,NaN,NaN,NaN,NaN
3,190001004925,2019,1506302,Salvaterra,15,PA,2,F,18,1.0,2,NaN,NaN,NaN,NaN,NaN
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50673,190006119978,2019,2930105,Senhor do Bonfim,29,BA,2,F,16,NaN,1,365.7,451.1,372.5,487.2,280.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0
50676,190006120091,2019,3155603,Rio Pardo de Minas,31,MG,3,M,17,1.0,2,459.8,439.3,448.7,509.6,440.0


## Salvar colunas selecionadas

In [134]:

df_selecao.to_csv('data/selecao.csv',index=False)
df_selecao

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
5,190001005059,2019,1506807,Santarém,15,PA,4,F,17,1.0,2,416.2,569.5,459.9,539.6,660.0
6,190001005138,2019,1508100,Tucuruí,15,PA,3,M,19,NaN,1,526.9,589.7,708.5,588.7,860.0
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,3,F,37,1.0,1,376.8,502.1,392.1,546.5,620.0
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,3,F,18,1.0,1,377.6,443.8,425.0,433.5,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,0,F,19,1.0,1,419.4,354.0,587.7,461.2,520.0
50672,190006119928,2019,2700300,Arapiraca,27,AL,3,F,26,1.0,1,567.1,513.7,551.0,559.2,460.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0


## Tirar alunos ocm nota 0 da selecao

In [143]:

notas_selecao = df_selecao.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0")
notas_selecao

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
5,190001005059,2019,1506807,Santarém,15,PA,4,F,17,1.0,2,416.2,569.5,459.9,539.6,660.0
6,190001005138,2019,1508100,Tucuruí,15,PA,3,M,19,NaN,1,526.9,589.7,708.5,588.7,860.0
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,3,F,37,1.0,1,376.8,502.1,392.1,546.5,620.0
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,3,F,18,1.0,1,377.6,443.8,425.0,433.5,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,0,F,19,1.0,1,419.4,354.0,587.7,461.2,520.0
50672,190006119928,2019,2700300,Arapiraca,27,AL,3,F,26,1.0,1,567.1,513.7,551.0,559.2,460.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0


In [140]:
# salvar a amostra filtrada em disco
notas_selecao.to_csv('data/enem19-sem_Zzero.csv',index=False)

In [136]:
#df_selecao = df_selecao[df_selecao["NU_NOTA_MT"] == 0]
#df_selecao

In [137]:
#df_selecao = df_selecao.drop(notas_zero.index)
#df_selecao

In [144]:
#Numero pessoas que sairam da analise
len(df_selecao)-len(notas_selecao)

27